# 💬 ChipChat - 데이터시트 챗봇

**전처리된 데이터시트 JSON 파일을 기반으로 질의응답을 수행하는 챗봇입니다.**

---

## 🚀 시작하기 전에

이 노트북은 Google Colab에서 개선된 ChipChat 앱을 실행하는 방법을 제공합니다.

⚠️ **중요**: 이 노트북은 반드시 **새로운 Colab 세션**에서 실행해주세요.
- 이전에 `prep_main.ipynb`를 실행했다면, 새로운 세션을 시작해주세요.
- 이는 의존성 충돌과 경로 문제를 방지하기 위함입니다.

## ✨ 새로운 기능들 (v2.0)
- 🤖 **AI Agent 시스템**: LangGraph 기반 스마트 에이전트
- 🔧 **3가지 Tool 자동 선택**: chipDB 검색, 벡터스토어 검색, PDF 처리
- 📊 **ChipDB.csv 연동**: 부품 사양 요약 데이터베이스 활용
- 🎯 **다중 LLM 지원**: OpenAI와 Claude 모델 선택 가능
- 📄 **실시간 PDF 업로드**: 새 데이터시트 자동 처리 및 통합
- 🔍 **고급 필터링**: 부품번호, 제조사, 카테고리별 검색
- 🛠️ **프롬프트 커스터마이징**: 시스템 프롬프트 자유 수정
- 🏷️ **메타데이터 추적**: 소스 정보 표시

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 전처리된 JSON 파일과 벡터 스토어를 관리합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# 데이터 폴더 설정
prep_json_folder = Path('/content/drive/MyDrive/prep_json')  # chipDB.csv 위치
vectorstore_folder = Path('/content/drive/MyDrive/vectorstore')
prompt_templates_folder = Path('/content/drive/MyDrive/prompt_templates')

# 필요한 폴더들 생성
vectorstore_folder.mkdir(parents=True, exist_ok=True)
prompt_templates_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 전처리 데이터 폴더: {prep_json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")
print(f"📁 프롬프트 템플릿 폴더: {prompt_templates_folder}")

# chipDB.csv 파일 확인
chipdb_file = prep_json_folder / 'chipDB.csv'
if chipdb_file.exists():
    # 파일 크기 확인 (적절한 단위로 표시)
    file_size_bytes = chipdb_file.stat().st_size
    if file_size_bytes < 1024:
        file_size_str = f"{file_size_bytes} bytes"
    elif file_size_bytes < 1024 * 1024:
        file_size_str = f"{file_size_bytes / 1024:.1f} KB"
    else:
        file_size_str = f"{file_size_bytes / 1024 / 1024:.1f} MB"
    
    print(f"\n📊 chipDB.csv 발견: {file_size_str}")
else:
    print(f"\n⚠️ chipDB.csv 파일이 없습니다: {chipdb_file}")

# Vectorstore 파일 확인 (하위 폴더 포함)
vectorstore_files = []
for ext in ['*.index', '*.faiss', '*.pkl', '*.parquet']:
    vectorstore_files.extend(list(vectorstore_folder.glob(f"**/{ext}")))

print(f"\n🗄️ 발견된 Vectorstore 파일: {len(vectorstore_files)}개")
vectorstore_folders = set()
for vs_file in vectorstore_files[:10]:  # 처음 10개만 표시
    relative_path = vs_file.relative_to(vectorstore_folder)
    vectorstore_folders.add(str(relative_path.parent))
    print(f" - {relative_path}")

if len(vectorstore_files) > 10:
    print(f" ... 외 {len(vectorstore_files) - 10}개")

# 발견된 vectorstore 폴더들 표시
if vectorstore_folders and vectorstore_folders != {'.'}:
    print(f"\n📁 Vectorstore 폴더들:")
    for folder in sorted(vectorstore_folders):
        if folder != '.':
            print(f" - {folder}")

# 상태 확인 및 경고
if not chipdb_file.exists() and not vectorstore_files:
    print("\n⚠️ chipDB.csv와 Vectorstore가 모두 없습니다.")
    print("💡 prep_main.ipynb를 먼저 실행하여 데이터를 전처리해주세요.")
elif not chipdb_file.exists():
    print("\n⚠️ chipDB.csv 파일이 없습니다. prep_main.ipynb로 chipDB.csv를 생성해주세요.")
elif not vectorstore_files:
    print("\n⚠️ Vectorstore가 없습니다. prep_main.ipynb로 Vectorstore를 생성해주세요.")
else:
    print("\n✅ chipDB.csv와 Vectorstore가 모두 준비되었습니다!")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
import subprocess
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

# GitHub 저장소 클론
try:
    if not Path('chipchat').exists():
        subprocess.run(['git', 'clone', 'https://github.com/doyoung42/chipchat_demo.git'], check=True)
        print("✅ GitHub 저장소 클론 완료")
    else:
        print("✅ GitHub 저장소 이미 존재")
except Exception as e:
    print(f"❌ GitHub 클론 실패: {str(e)}")

# 디렉토리 이동
os.chdir('chipchat_demo')

# requirements.txt 설치
try:
    subprocess.run(['pip', 'install', '-r', 'requirements.txt', '-q'], check=True)
    print("✅ Requirements 설치 완료")
except Exception as e:
    print(f"❌ Requirements 설치 실패: {str(e)}")

# 추가 패키지 설치 (Colab 전용)
try:
    subprocess.run(['pip', 'install', 'pyngrok==7.0.1', '-q'], check=True)
    print("✅ pyngrok 설치 완료")
except Exception as e:
    print(f"❌ pyngrok 설치 실패: {str(e)}")

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 새로운 디렉토리 구조 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    for subdir in ['config', 'models', 'utils', 'app']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("\n✅ 라이브러리 설치 및 경로 설정 완료!")
print("\n📋 지원되는 LLM 모델:")
print("🔸 OpenAI: gpt-4o-mini, gpt-4o, gpt-3.5-turbo")
print("🔸 Claude: claude-3-sonnet, claude-3-haiku, claude-3-opus")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown AI 서비스의 API 키를 입력하여 챗봇을 설정합니다.

import os
import json

openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # 환경 변수 설정
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("✅ OpenAI API 키 설정 완료!")
        
    if claude_api_key:
        os.environ["ANTHROPIC_API_KEY"] = claude_api_key
        print("✅ Claude API 키 설정 완료!")
    
    if hf_token:
        os.environ["HF_TOKEN"] = hf_token
        print("✅ HuggingFace 토큰 설정 완료!")
    
    # Streamlit 시크릿 파일 생성
    secrets_dir = Path(".streamlit")
    secrets_dir.mkdir(exist_ok=True)
    
    secrets = {}
    if openai_api_key:
        secrets["openai_api_key"] = openai_api_key
    if claude_api_key:
        secrets["anthropic_api_key"] = claude_api_key
    if hf_token:
        secrets["hf_token"] = hf_token
    
    with open(secrets_dir / "secrets.toml", "w") as f:
        for key, value in secrets.items():
            f.write(f'{key} = "{value}"\n')
    
    # 토큰 저장
    try:
        from src.config.token_manager import TokenManager
        token_manager = TokenManager()
        
        if openai_api_key:
            token_manager.set_token('openai', openai_api_key)
        
        if claude_api_key:
            token_manager.set_token('anthropic', claude_api_key)
        
        if hf_token:
            token_manager.set_token('huggingface', hf_token)
            
        print("✅ 토큰 관리자에 API 키 저장 완료!")
    except Exception as e:
        print(f"❌ 토큰 저장 중 오류가 발생했습니다: {str(e)}")
    
    # 기본 프롬프트 템플릿 생성
    default_template = {
        "pre": "당신은 전자 부품 데이터시트에 대해 응답하는 도우미입니다. 제공된 컨텍스트 정보를 기반으로 질문에 답변하세요.",
        "post": "검색된 정보를 바탕으로 명확하고 간결하게 답변해주세요."
    }
    
    template_file = prompt_templates_folder / "default_template.json"
    if not template_file.exists():
        with open(template_file, 'w', encoding='utf-8') as f:
            json.dump(default_template, f, ensure_ascii=False, indent=2)
            
        print("✅ 기본 프롬프트 템플릿이 생성되었습니다.")

In [ ]:
#@title 📊 4단계: Streamlit 서버 실행 { display-mode: "form" }
#@markdown Streamlit 서비스를 실행하여 웹 인터페이스를 제공합니다.

import subprocess
import threading
import time
import os
import json
from pathlib import Path

# 환경 변수 설정
os.environ['VECTORSTORE_PATH'] = str(vectorstore_folder)
os.environ['JSON_FOLDER_PATH'] = str(prep_json_folder)  # chipDB.csv가 있는 폴더
os.environ['PROMPT_TEMPLATES_PATH'] = str(prompt_templates_folder)

# Streamlit 실행 함수
def run_streamlit():
    cmd = [
        "streamlit", "run", 
        "src/app/streamlit_app.py",
        "--server.port=8501", 
        "--server.address=0.0.0.0",  # 외부 접속 허용
        "--server.headless=true",
        "--browser.serverAddress=localhost",
        "--browser.gatherUsageStats=false"
    ]
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    return process

# ngrok 사용 여부 선택
use_ngrok = False #@param {type:"boolean"}
ngrok_token = "" #@param {type:"string"}

# Streamlit 서버 실행
print("🚀 Streamlit 서버를 시작합니다...")
process = run_streamlit()
time.sleep(8)  # 서버 시작 대기

if use_ngrok and ngrok_token:
    # ngrok 사용 (외부 접속 가능한 공개 URL)
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(8501).public_url
        print(f"\n✅ 서버가 시작되었습니다! (ngrok 사용)")
        print(f"\n🔗 다음 URL을 통해 ChipChat에 접속할 수 있습니다:")
        print(f"📱 {public_url}")
        print("\n이 URL은 세션이 유지되는 동안에만 유효합니다.")
    except Exception as e:
        print(f"\n❌ ngrok 연결 실패: {str(e)}")
        print("Google Colab 내장 기능을 사용합니다.")
        use_ngrok = False

if not use_ngrok:
    # Google Colab 내장 기능 사용 (권장)
    print(f"\n✅ 서버가 시작되었습니다! (Colab 내장 기능 사용)")
    print(f"\n🔗 ChipChat에 접속하는 방법:")
    print(f"1️⃣ 포트 8501에서 실행 중입니다")
    print(f"2️⃣ 아래 버튼을 클릭하거나, 왼쪽 사이드바에서 '포트' 탭을 확인하세요")
    
    # Colab의 포트 포워딩 기능 활용
    from google.colab import output
    output.serve_kernel_port_as_window(8501)

print(f"\n💡 팁: 앱이 로딩되는데 몇 초 정도 걸릴 수 있습니다.")

In [ ]:
#@title 🛑 5단계: 서버 중지 { display-mode: "form" }
#@markdown 작업을 마치면 서버를 중지합니다.

import os
import signal
import subprocess

# 서버 중지 여부 확인
stop_server = True  #@param {type:"boolean"}

if stop_server:
    # ngrok 터널 종료 (ngrok 사용한 경우에만)
    try:
        from pyngrok import ngrok
        ngrok.kill()
        print("✅ ngrok 터널이 종료되었습니다.")
    except ImportError:
        # pyngrok이 import되지 않은 경우 (정상적인 상황)
        pass
    except Exception as e:
        # ngrok 관련 다른 오류
        print(f"ℹ️ ngrok 종료: {str(e)}")
    
    # Streamlit 프로세스 종료
    try:
        # 모든 streamlit 프로세스 종료
        subprocess.run(['pkill', '-f', 'streamlit'], check=False)
        print("✅ Streamlit 서버가 종료되었습니다.")
    except Exception as e:
        print("⚠️ Streamlit 서버 종료 중 오류가 발생했습니다.")
        print("💡 수동으로 런타임을 재시작하여 서버를 종료할 수 있습니다.")
    
    print("\n🔄 런타임을 재시작하면 모든 프로세스가 완전히 종료됩니다.")

---

## 🛠️ 문제 해결

**애플리케이션이 실행되지 않는 경우:**

1. **API 키 확인**: OpenAI API 키가 올바르게 입력되었는지 확인
2. **세션 재시작**: 런타임 → 세션 재시작 후 처음부터 다시 실행
3. **JSON 파일 확인**: 전처리된 JSON 파일이 Google Drive에 존재하는지 확인

**사용 중 문제가 발생하는 경우:**
- 브라우저를 새로고침하세요
- 네트워크 연결 상태를 확인하세요

---

## 📞 지원

추가 도움이 필요하시면 GitHub 이슈를 등록해주세요:
🔗 https://github.com/doyoung42/chipchat/issues
